In [ ]:
source extend_bashkernel.source

In [ ]:
# (1) optional reset of Wakame-vdc
# Comment out the following lines and run cell to remove all
# added images and instances.

# reset-wakame.sh

In [ ]:
# (2) Create Centos instance using the Wakame-vdc GUI

# If needed, one of these can
# be entered on the next line: (hint, restore, or check)
# restore

In [ ]:
# (3) Wait for instance's ssh port to become active
: ${IP:=10.0.2.100}
while [[ "$(echo | nc -w 1 "$IP" 22)" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port 22"

## repoファイルのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 curl -fSkL http://pkg.jenkins-ci.org/redhat/jenkins.repo -o /etc/yum.repos.d/jenkins.repo

## 公開鍵のインポート

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key

## java実行環境のインストール

In [ ]:
# This installs the Oracle version of Java.  The RPM file was
# already downloaded to /home/centos/notebooks/.downloads

cd /home/centos/notebooks/.downloads
tar c jdk-8u73-linux-x64.rpm | ssh -qi /home/centos/mykeypair root@10.0.2.100 tar xv

ssh -qi /home/centos/mykeypair root@10.0.2.100 <<EOS

rpm -ivh jdk-8u73-linux-x64.rpm

cat <<CFG >/etc/profile.d/java.sh
JAVA_HOME=/usr/java/jdk1.8.0_25/
PATH=$JAVA_HOME/bin:$PATH
export PATH JAVA_HOME
export CLASSPATH=.
CFG

EOS

## jenkins coreのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y jenkins

## 関連するビルドツールのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y \
    git \
    iputils nc \
    qemu-kvm qemu-img \
    parted kpartx \
    rpm-build automake createrepo \
    openssl-devel zlib-devel readline-devel \
    gcc

## sudo設定

### sudo権限の付与


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 "echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers"
ssh -qi ../mykeypair root@10.0.2.100 grep jenkins /etc/sudoers

### tty設定の変更

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 'sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers'

### Loosen Java Security so Jenkins Plugins will work

In [ ]:
# see https://issues.jenkins-ci.org/browse/JENKINS-31089
javadir=/usr/lib/jvm/java-1.7.0-openjdk-1.7.0.95.x86_64/
secfile=jre/lib/security/java.security
ssh -qi ../mykeypair root@10.0.2.100 \
   "sudo sed -i 's/, MD5, RSA keySize < 1024//' $javadir/$secfile"


## jenkinsの起動


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 service jenkins start